In [16]:
import pandas as pd
from db import Company, CoalCompanyPerformance
import re
from tabulate import tabulate

In [17]:
companies = Company.select()
for company in companies:
    pass

ccp = CoalCompanyPerformance.select()
for c in ccp:
    pass

In [18]:
existing_names = [re.sub(r'\b(PT|Tbk)\b', '', company.name).lower().strip() for company in companies]

In [19]:
e_company_df = pd.read_csv("Sample db - company_directory.csv")
e_company_df.loc[e_company_df['company_type'] == "Owner", 'company_type'] = "Mine Owner"
e_company_df

,id,name,idx_ticker,parent company id,operation_province,operation_kabkot,representative_address,company_type,key_operation,activities,website,phone_number,email,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,1,Bayan Resources Tbk,BYAN.JK,NaN,Kalimantan Timur,NaN,"Gedung Office 8, Lantai 37 Unit A-H Jl. Senopa...",Holding,Mining,NaN,www.bayan.com.sg,021-29356888,corporate.secretary@bayan.com.sg,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,PT Fajar Sakti Prima,NaN,1.0,Kalimantan Timur,Kutai Kartanegara,NaN,Mine Owner,Mining,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,PT Bara Tabang,NaN,1.0,Kalimantan Timur,Kutai Kartanegara,NaN,Mine Owner,Mining,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,PT Tiwa Abadi,NaN,1.0,Kalimantan Timur,Kutai Kartanegara,NaN,Mine Owner,Mining,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,PT Tanur Jaya,NaN,1.0,Kalimantan Timur,Kutai Kartanegara,NaN,Mine Owner,Mining,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,125,PT Karimata Multi Prima,NaN,50.0,NaN,NaN,NaN,Trader,Coal Trading,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,126,PT Inti Buana Mining,NaN,50.0,NaN,NaN,NaN,Trader,Coal Trading,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126,127,PT Nusantara Mapan Lestari,NaN,50.0,NaN,NaN,NaN,Trader,Coal Trading,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,128,PT Hanson Energi Baturaja,NaN,50.0,NaN,NaN,NaN,Trader,Coal Trading,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
excluded_companies = ("PT BUMA Internasional Grup Tbk", "PT Murawai Coal")

In [21]:
e_company_df = e_company_df[~e_company_df["name"].isin(excluded_companies)]

In [22]:
e_company_df['name_cleaned'] = e_company_df['name'].str.replace(r'\b(PT|Tbk)\b', '', regex=True).str.lower().str.strip()
new_company_df = e_company_df[~e_company_df['name_cleaned'].isin(existing_names)]
new_company_df

C:\Users\lukas\AppData\Local\Temp\ipykernel_15228\915390499.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e_company_df['name_cleaned'] = e_company_df['name'].str.replace(r'\b(PT|Tbk)\b', '', regex=True).str.lower().str.strip()


,id,name,idx_ticker,parent company id,operation_province,operation_kabkot,representative_address,company_type,key_operation,activities,...,phone_number,email,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,name_cleaned
128,129,PT Rimba Perkasa Utama,NaN,NaN,NaN,NaN,NaN,Contractor,Mining Services,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rimba perkasa utama


In [23]:
def safe_value(val):
    return None if pd.isna(val) else val

for _, row in new_company_df.iterrows():

    company = Company(
        name=safe_value(row['name']),
        idx_ticker=safe_value(row['idx_ticker']),
        operation_province=safe_value(row['operation_province']),
        operation_kabkot=safe_value(row['operation_kabkot']),
        representative_address=safe_value(row['representative_address']),
        company_type=safe_value(row['company_type']),
        key_operation=safe_value(row['key_operation']),
        activities=safe_value(row['activities']),
        website=safe_value(row['website']),
        phone_number=safe_value(row['phone_number']),
        email=safe_value(row['email']),
    )
    company.save()


In [25]:
existing_company_df = e_company_df[e_company_df['name_cleaned'].isin(existing_names)]
existing_company_df

,id,name,idx_ticker,parent company id,operation_province,operation_kabkot,representative_address,company_type,key_operation,activities,...,phone_number,email,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,name_cleaned
0,1,Bayan Resources Tbk,BYAN.JK,NaN,Kalimantan Timur,NaN,"Gedung Office 8, Lantai 37 Unit A-H Jl. Senopa...",Holding,Mining,NaN,...,021-29356888,corporate.secretary@bayan.com.sg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bayan resources
1,2,PT Fajar Sakti Prima,NaN,1.0,Kalimantan Timur,Kutai Kartanegara,NaN,Mine Owner,Mining,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fajar sakti prima
2,3,PT Bara Tabang,NaN,1.0,Kalimantan Timur,Kutai Kartanegara,NaN,Mine Owner,Mining,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bara tabang
3,4,PT Tiwa Abadi,NaN,1.0,Kalimantan Timur,Kutai Kartanegara,NaN,Mine Owner,Mining,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tiwa abadi
4,5,PT Tanur Jaya,NaN,1.0,Kalimantan Timur,Kutai Kartanegara,NaN,Mine Owner,Mining,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tanur jaya
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,124,PT Aquela Pratama Indonesia,NaN,50.0,NaN,NaN,NaN,Trader,Investment,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,aquela pratama indonesia
124,125,PT Karimata Multi Prima,NaN,50.0,NaN,NaN,NaN,Trader,Coal Trading,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,karimata multi prima
125,126,PT Inti Buana Mining,NaN,50.0,NaN,NaN,NaN,Trader,Coal Trading,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inti buana mining
126,127,PT Nusantara Mapan Lestari,NaN,50.0,NaN,NaN,NaN,Trader,Coal Trading,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nusantara mapan lestari


In [32]:
fields_to_compare = [
    'name',
    'idx_ticker',
    'operation_province',
    'operation_kabkot',
    'representative_address',
    'company_type',
    'key_operation',
    'activities',
    'website',
    'phone_number',
    'email'
]

for _, row in existing_company_df.iterrows():
    company = Company.get_or_none(Company.name == row['name'])
    if company and company.name not in ("PT Pada Idi"):
        differences = []
        for field in fields_to_compare:
            model_value = getattr(company, field)
            df_value = safe_value(row[field])

            if (model_value != df_value) and (df_value is not None):
                differences.append([field, model_value, df_value])
                # setattr(company, field, df_value) 

        if differences:
            # company.save()
            print(f"\nDifferences for company '{company.name}':")
            print(tabulate(differences, headers=["Field", "DB Value", "CSV Value"], tablefmt="grid"))


Differences for company 'PT Mandiri Herindo Adiperkasa Tbk':
+---------------+-----------------+------------------------------+
| Field         | DB Value        | CSV Value                    |
+===============+=================+==============================+
| key_operation | Mining Services | Overburden Removal & Hauling |
+---------------+-----------------+------------------------------+

Differences for company 'PT Diva Kencana Borneo':
+------------------+-------------+-------------+
| Field            | DB Value    | CSV Value   |
+==================+=============+=============+
| operation_kabkot | Kutai Barat | West Kutai  |
+------------------+-------------+-------------+

Differences for company 'PT Karya Borneo Agung':
+------------------+-------------+-------------+
| Field            | DB Value    | CSV Value   |
+==================+=============+=============+
| operation_kabkot | Kutai Barat | West Kutai  |
+------------------+-------------+-------------+

Differences